In [0]:
%pip install -qqqq -U unitycatalog-ai[databricks]
dbutils.library.restartPython()

In [0]:
%load_ext autoreload
%autoreload 2
import warnings

warnings.simplefilter(action="ignore")

In [0]:
dbutils.widgets.text(name="catalog", defaultValue="bo_cheng_dnb_demos", label="catalog")
dbutils.widgets.text(name="schema", defaultValue="agents", label="schema")

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

CATALOG = dbutils.widgets.get("catalog")
SCHEMA = dbutils.widgets.get("schema")

In [0]:
# Define a function that requires an external PyPI dependency


def plot_to_json(payload_json: str) -> str:
    """
    A function to plot a line chart and return the JSON representation of the chart.

    Args:
        payload_json (str): JSON string with shape {"rows": [{"x": <float>, "y": <float>}, ...]}

    Returns:
        A string json representation of the plotly graph
    """

    import plotly.express as px  # NOTE that you must still import the library to use within the function.
    import pandas as pd
    import json

    obj = json.loads(payload_json)
    rows = obj.get("rows")
    df = pd.DataFrame(rows)
    fig = px.line(df, x="x", y="y", title="Unsorted Input")

    return str(fig.to_json())


# Create the function and supply the dependency in standard PyPI format
client.create_python_function(
    func=plot_to_json,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
    dependencies=["plotly", "pandas"],
)

In [0]:
my_callable = client.get_function_as_callable(
    function_name=f"{CATALOG}.{SCHEMA}.plot_to_json"
)

In [0]:
import plotly.io as pio

json_string = my_callable(payload_json='{"rows": [{"x": 1, "y": 1}, {"x": 2, "y": 2}]}')
fig = pio.from_json(json_string)
fig.show()